# Create Tracks

Create tracks from passed inner and outer borders. The goal is to automatically generate code that represents the track, i.e., the set of bad states, waypoints as well as the bounding box.


In [1]:
in_inner = '../racetracks/austria/austria_inner.csv' # filename for the file containing the vertices
in_outer = '../racetracks/austria/austria_outer.csv' # filename for the file containing the vertices
inner_track_name = 'inner_track' # name for the inner track
inner_track_filename = inner_track_name + '.poly' # filename which contains input for triangle of the inner part of the track
outer_track_name = 'outer_track' # name for the inner track
outer_track_filename = outer_track_name + '.poly' # filename which contains input for triangle of the inner part of the track
project_base = '/home/bmaderbacher/Documents/projects/simplex_synthesis/repos/simplex-architectures/' # project base folder 
triangle_folder = project_base + 'cmake-build-release/triangle-prefix/src/triangle/' # triangle-executable folder
hypro_outfile = project_base + 'racetracks/austria/bad_states.h' # the file will contain definitions of polytopes for the inner part of the track
hypro_playground = project_base + 'racetracks/austria/playground.h' # contains the bounding box
hypro_waypoints = project_base + 'racetracks/austria/waypoints.h' # holds definitions of waypoints
hypro_segments = project_base + 'racetracks/austria/segments.h' # will hold the definitions of the track segments

Read in the csv-file containing the vertices of the track.

In [2]:
inner_nodes = []
outer_nodes = []

with open(in_inner, 'r') as f:
    inner_nodes = [ (line.split(",")[0].strip(), line.split(",")[1].strip()) for line in f.readlines() if '#' not in line]

with open(in_outer, 'r') as f:
    outer_nodes = [ (line.split(",")[0].strip(), line.split(",")[1].strip()) for line in f.readlines() if '#' not in line]


Also compute the corners of the bounding box

In [3]:
minx = miny = float('inf')
maxx = maxy = float('-inf')
for v in outer_nodes:
    minx = min(minx, float(v[0]))
    maxx = max(maxx, float(v[0]))
    miny = min(miny, float(v[1]))
    maxy = max(maxy, float(v[1]))

# add a bit of margin for stability
minx -= 1
maxx += 1
miny -= 1
maxy += 1

In [4]:
from platform import node


def writePolyFile(filename,nodes,holes,bounding_box):
    out = open(filename,'w')
    # write first line: number of nodes, dimension, number of attributes, markers (1 or 0)
    out.write(str(len(nodes)+len(bounding_box)) + " 2 0 0\n")
    # write all nodes
    for i in range(len(nodes)):
        out.write(str(i) + " " + str(nodes[i][0]) + " " + str(nodes[i][1]) + "\n")

    for i in range(len(bounding_box)):
        out.write(str(i+len(nodes)) + " " + str(bounding_box[i][0]) + " " + str(bounding_box[i][1]) + "\n")

    # now add segments, first a line indicating how many segments there will be, whether we want to have boundary markers, 
    # then the segments (the two node indices defining a segment)
    out.write(str(len(nodes)+len(bounding_box)) + " 0\n")
    for i in range(len(nodes)):
        out.write(str(i) + " " + str(i) + " " + str((i+1) % len(nodes)) + "\n")
    
    # write segments for bounding box manually, if we have a bounding box
    if len(bounding_box) > 0:
        out.write(str(len(nodes)) + " " + str(len(nodes)) + " " + str((len(nodes)+1)) + "\n")
        out.write(str(len(nodes)+1) + " " + str(len(nodes)+1) + " " + str((len(nodes)+2)) + "\n")
        out.write(str(len(nodes)+2) + " " + str(len(nodes)+2) + " " + str((len(nodes)+3)) + "\n")
        out.write(str(len(nodes)+3) + " " + str(len(nodes)+3) + " " + str((len(nodes))) + "\n")

    # finally, add one line for indicating the number of holes (with no boundary markers)
    out.write(str(len(holes)) + " 0\n")
    # if there are holes, give inner points
    for i in range(len(holes)):
        out.write(str(i) + " " + str(holes[i][0]) + " " + str(holes[i][1]) + "\n")

    out.close()

Prepare a *.poly file for triangle which contains all the nodes and segments, one for the inner and one for the outer track. Note that for the outer track we need to define a hole (the inner track). To do so, we first compute the inner track triangulation, then take a point which is definitely inside one of the computed triangles (the mid-point of the first triangle) and use this point to define the inside of the hole for the outer triangulation. Note that we also need to provide vertices of the bounding box for the outer set.

In [5]:
import os

# write file for inner track part
writePolyFile(inner_track_filename,inner_nodes, [], [])
# call to triangle
os.system(triangle_folder + "triangle -pYj " + inner_track_filename)
# collect triangles
inner_vertex_sets = []
with open(inner_track_name + ".1.ele") as f:
    inner_vertex_sets = [ (int(line.split()[1]), int(line.split()[2]), int(line.split()[3])) for line in f.readlines() if len(line.split()) == 4]

print(str(inner_vertex_sets))

# For the outer track we need to define a hole. A hole is given as a point (inside the hole) surrounded by segments
# We already have the segments, to get an internal point, we try to get an internal point of one of the polytopes
# of the inner set and use this as the basis for our hole (there is only one). To get an inner point, take the average of the first triangle
holes = [ ((float(inner_nodes[inner_vertex_sets[0][0]][0]) + float(inner_nodes[inner_vertex_sets[0][1]][0]) + float(inner_nodes[inner_vertex_sets[0][2]][0]))/3, (float(inner_nodes[inner_vertex_sets[0][0]][1]) + float(inner_nodes[inner_vertex_sets[0][1]][1]) + float(inner_nodes[inner_vertex_sets[0][2]][1]))/3) ]
# collect vertices of the bounding box
bbox = [ (minx,miny),(maxx,miny),(maxx,maxy),(minx,maxy) ]
# write file with holes
writePolyFile(outer_track_filename,outer_nodes, holes, bbox)
# call triangle
os.system(triangle_folder + "triangle -pYj " + outer_track_filename)
# collect triangles
outer_vertex_sets = []
with open(outer_track_name + ".1.ele") as f:
    outer_vertex_sets = [ (int(line.split()[1]), int(line.split()[2]), int(line.split()[3])) for line in f.readlines() if len(line.split()) == 4]

print(str(outer_vertex_sets))


Opening inner_track.poly.
Constructing Delaunay triangulation by divide-and-conquer method.
Delaunay milliseconds:  0
Recovering segments in Delaunay triangulation.
Segment milliseconds:  0
Removing unwanted triangles.
Hole milliseconds:  0

Writing inner_track.1.node.
Writing inner_track.1.ele.
Writing inner_track.1.poly.

Output milliseconds:  1
Total running milliseconds:  1

Statistics:

  Input vertices: 30
  Input segments: 30
  Input holes: 0

  Mesh vertices: 30
  Mesh triangles: 28
  Mesh edges: 57
  Mesh exterior boundary edges: 30
  Mesh interior boundary edges: 0
  Mesh subsegments (constrained edges): 30

[(9, 26, 25), (26, 9, 8), (9, 25, 10), (24, 11, 25), (25, 11, 10), (23, 11, 24), (27, 26, 29), (29, 28, 27), (26, 0, 29), (7, 6, 1), (7, 26, 8), (1, 0, 7), (6, 2, 1), (7, 0, 26), (23, 22, 21), (21, 20, 11), (12, 11, 20), (21, 11, 23), (19, 18, 17), (17, 16, 19), (20, 19, 14), (16, 14, 19), (12, 20, 13), (3, 2, 5), (6, 5, 2), (13, 20, 14), (4, 3, 5), (16, 15, 14)]
Opening 

Create HyPro-code that creates polytopes for each of the passed triangles.

In [6]:
# add vertices of the bounding box also to the outer nodex, since they will be used later for the file writing
outer_nodes.append((minx,miny))
outer_nodes.append((maxx,miny))
outer_nodes.append((maxx,maxy))
outer_nodes.append((minx,maxy))

In [7]:
def createPolytopeCode(nodes,vertices):
	print("Have vertex indices " + str(vertices))
	return '{\n\
	\tstd::vector<hypro::Point<Number>> points{\n\
	\thypro::Point<Number>({' + str(nodes[vertices[0]][0]) + ',' + str(nodes[vertices[0]][1]) +'}),\n\
	\thypro::Point<Number>({' + str(nodes[vertices[1]][0]) + ',' + str(nodes[vertices[1]][1]) +'}),\n\
	\thypro::Point<Number>({' + str(nodes[vertices[2]][0]) + ',' + str(nodes[vertices[2]][1]) +'}),\n\
	};\n\
	hypro::HPolytope<Number> poly{points};\n\
	res.push_back(hypro::Condition<Number>(poly.matrix(), poly.vector()));\n\
	}'

In [8]:
hyprofile = open(hypro_outfile,'w')

hyprofile.write(
'#include <hypro/representations/GeometricObjectBase.h>\n\
#include <vector>\n\n\
#pragma once\n\n\
namespace simplexArchitectures {\n\
template<typename Automaton>\n\
typename Automaton::conditionVector createBadStates() {\n\
\tusing Number = typename Automaton::NumberType;\n\
\tauto res = typename Automaton::conditionVector();\n\n')

for v_set in inner_vertex_sets:
    hyprofile.write(createPolytopeCode(inner_nodes,v_set))

print(str(len(outer_nodes)))
for v_set in outer_vertex_sets:
    print(str(v_set))
    hyprofile.write(createPolytopeCode(outer_nodes,v_set))
    
hyprofile.write('\n\treturn res;\n}\n} // namespace\n')

hyprofile.close()

Have vertex indices (9, 26, 25)
Have vertex indices (26, 9, 8)
Have vertex indices (9, 25, 10)
Have vertex indices (24, 11, 25)
Have vertex indices (25, 11, 10)
Have vertex indices (23, 11, 24)
Have vertex indices (27, 26, 29)
Have vertex indices (29, 28, 27)
Have vertex indices (26, 0, 29)
Have vertex indices (7, 6, 1)
Have vertex indices (7, 26, 8)
Have vertex indices (1, 0, 7)
Have vertex indices (6, 2, 1)
Have vertex indices (7, 0, 26)
Have vertex indices (23, 22, 21)
Have vertex indices (21, 20, 11)
Have vertex indices (12, 11, 20)
Have vertex indices (21, 11, 23)
Have vertex indices (19, 18, 17)
Have vertex indices (17, 16, 19)
Have vertex indices (20, 19, 14)
Have vertex indices (16, 14, 19)
Have vertex indices (12, 20, 13)
Have vertex indices (3, 2, 5)
Have vertex indices (6, 5, 2)
Have vertex indices (13, 20, 14)
Have vertex indices (4, 3, 5)
Have vertex indices (16, 15, 14)
34
(26, 30, 25)
Have vertex indices (26, 30, 25)
(25, 30, 24)
Have vertex indices (25, 30, 24)
(7, 8, 9

Create header encoding the road segments

In [9]:
def createSegments(inner_nodes,outer_nodes,clockwise):
    assert(len(inner_nodes) == len(outer_nodes))
    returnstring = 'std::vector<Segment>{\n'
    if clockwise:
        first = True
        for i in range(len(inner_nodes)):
            if first:
                first = False
            else:
                returnstring += ",\n"
            returnstring += f'\t{{ hypro::Point<Number>{{ {outer_nodes[i][0]} , {outer_nodes[i][1]} }} , \
hypro::Point<Number>{{ {inner_nodes[i][0]}, {inner_nodes[i][1]} }}, \
hypro::Point<Number>{{ {outer_nodes[(i+1)%len(outer_nodes)][0]}, {outer_nodes[(i+1)%len(outer_nodes)][1]} }}, \
hypro::Point<Number>{{ {inner_nodes[(i+1)%len(inner_nodes)][0]}, {inner_nodes[(i+1)%len(inner_nodes)][1]} }} }}'
    else:
        first = True
        for i in range(len(inner_nodes)):
            if first:
                first = False
            else:
                returnstring += ",\n"
            returnstring += f'\t{{ hypro::Point<Number>{{ {inner_nodes[i][0]} , {inner_nodes[i][1]} }} , \
hypro::Point<Number>{{ {outer_nodes[i][0]}, {outer_nodes[i][1]} }}, \
hypro::Point<Number>{{ {inner_nodes[(i+1)%len(inner_nodes)][0]}, {inner_nodes[(i+1)%len(inner_nodes)][1]} }}, \
hypro::Point<Number>{{ {outer_nodes[(i+1)%len(outer_nodes)][0]}, {outer_nodes[(i+1)%len(outer_nodes)][1]} }} }}'

    returnstring += '};'
    return returnstring

hyprofile = open(hypro_segments,'w')

hyprofile.write(
'#include <hypro/datastructures/Point.h>\n\
#include <vector>\n\n\
#pragma once\n\n\
namespace simplexArchitectures {\n\
template<typename Segment>\n\
std::vector<Segment> createSegments() {\n\
\tauto res = ')
# remove the last 4 nodes from the outer nodes, since those were used for the bounding box earlier
hyprofile.write(createSegments(inner_nodes,outer_nodes[:-4],True))
    
hyprofile.write('\n\treturn res;\n}\n} // namespace\n')

hyprofile.close()

Create HyPro-code defining the waypoints in the middle of the track

In [10]:
def createWaypoints(inner_nodes,outer_nodes):
    assert(len(inner_nodes) == len(outer_nodes))
    returnstring = 'std::vector<hypro::Point<Number>>{\n'
    first = True
    for i in range(len(inner_nodes)):
        midx = min(float(outer_nodes[i][0]),float(inner_nodes[i][0])) + abs(float(outer_nodes[i][0]) - float(inner_nodes[i][0]))/2.0
        midy = min(float(outer_nodes[i][1]),float(inner_nodes[i][1])) + abs(float(outer_nodes[i][1]) - float(inner_nodes[i][1]))/2.0
        if first:
            first = False
        else:
            returnstring += ",\n"
        returnstring += f'\thypro::Point<Number>{{ {midx} , {midy} }}'

    returnstring += '};'
    return returnstring

hyprofile = open(hypro_waypoints,'w')

hyprofile.write(
'#include <hypro/datastructures/Point.h>\n\
#include <vector>\n\n\
#pragma once\n\n\
namespace simplexArchitectures {\n\
template<typename Number>\n\
std::vector<hypro::Point<Number>> createWaypoints() {\n\
\tauto res = ')
# remove the last 4 nodes from the outer nodes, since those were used for the bounding box earlier
hyprofile.write(createWaypoints(inner_nodes,outer_nodes[:-4]))
    
hyprofile.write('\n\treturn res;\n}\n} // namespace\n')

hyprofile.close()

Create HyPro-code that determines the track playground (i.e., the bounding box)

In [11]:
hyprofile = open(hypro_playground,'w')

hyprofile.write(
'#include <hypro/representations/GeometricObjectBase.h>\n\
#include <vector>\n\n\
#pragma once\n\n\
namespace simplexArchitectures {\n\
template<typename Number>\n\
typename hypro::Box<Number> createPlayground() {\n\
\tusing I = carl::Interval<Number>;\n\
\tusing IV = std::vector<I>;\n\
\treturn hypro::Box<Number>{IV{I{'+str(minx)+","+str(maxx)+'},I{'+str(miny)+','+str(maxy)+'}}};\n}'
)
hyprofile.write('\n} // namespace\n')

hyprofile.close()